In [1]:
!pip install -q dask[complete] plotly==5.13.0 dask-ml

import random
import dask
import dask.dataframe as dd
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dask_ml.preprocessing import StandardScaler
from dask_ml.cluster import KMeans

# Enhanced Data Generation with Realistic Progression
def generate_data(player_id):
    """Create dynamic Valorant match data with rank progression"""
    agents = {
        'Duelist': ['Jett', 'Reyna', 'Phoenix', 'Raze', 'Neon'],
        'Controller': ['Brimstone', 'Viper', 'Omen', 'Astra'],
        'Sentinel': ['Sage', 'Cypher', 'Killjoy', 'Chamber'],
        'Initiator': ['Sova', 'Breach', 'Skye', 'KAY/O', 'Fade']
    }

    maps = ['Bind', 'Haven', 'Split', 'Ascent', 'Icebox', 'Breeze', 'Fracture']
    ranks = [
        'Iron 1', 'Iron 2', 'Iron 3', 'Bronze 1', 'Bronze 2', 'Bronze 3',
        'Silver 1', 'Silver 2', 'Silver 3', 'Gold 1', 'Gold 2', 'Gold 3',
        'Platinum 1', 'Platinum 2', 'Platinum 3', 'Diamond 1', 'Diamond 2',
        'Diamond 3', 'Immortal 1', 'Immortal 2', 'Immortal 3', 'Radiant'
    ]

    data = []
    current_rank_idx = 8  # Starting at Silver 1
    rank_history = []

    for match_num in range(200):
        role = random.choice(list(agents.keys()))
        agent = random.choice(agents[role])
        map_choice = random.choice(maps)

        # Dynamic performance based on rank progression
        rank_modifier = current_rank_idx / len(ranks)
        kills = max(0, int(np.random.normal(15 + 5*rank_modifier, 3)))
        deaths = max(1, int(np.random.normal(12 - 3*rank_modifier, 2)))
        assists = max(0, int(np.random.normal(8 + 2*rank_modifier, 2)))

        combat_score = kills*4.5 + assists*1.5 + random.randint(-20, 20)
        headshot = np.clip(np.random.normal(25 + 10*rank_modifier, 5), 10, 45)

        # Rank progression logic
        win_prob = 0.5 + (current_rank_idx/len(ranks))*0.3
        outcome = 'Win' if random.random() < win_prob else 'Loss'
        current_rank_idx = np.clip(
            current_rank_idx + (1 if outcome == 'Win' else -1) + random.choice([-1,0,1]),
            0, len(ranks)-1
        )
        rank_history.append(current_rank_idx)

        data.append({
            'PlayerID': player_id,
            'Agent': agent,
            'Role': role,
            'Map': map_choice,
            'Rank': ranks[current_rank_idx],
            'Kills': kills,
            'Deaths': deaths,
            'Assists': assists,
            'CombatScore': combat_score,
            'Headshot%': round(headshot, 1),
            'Outcome': outcome,
            'Date': pd.date_range('2024-01-01', periods=200)[match_num].strftime('%Y-%m-%d')
        })

    return pd.DataFrame(data)

# Create Dask DataFrame
player_id = input("Enter your Valorant ID (e.g., Tenz#NA1): ")
ddf = dd.from_pandas(generate_data(player_id), npartitions=4)

# Feature Engineering
ddf['KDA'] = (ddf['Kills'] + ddf['Assists']) / ddf['Deaths'].where(ddf['Deaths'] != 0, 1)
ddf['ImpactScore'] = ddf['CombatScore'] * ddf['KDA']
ddf['ADR'] = (ddf['CombatScore'] / 4.5).round(1)
ddf['RankScore'] = ddf['Rank'].map({rank: idx for idx, rank in enumerate([
    'Iron 1', 'Iron 2', 'Iron 3', 'Bronze 1', 'Bronze 2', 'Bronze 3',
    'Silver 1', 'Silver 2', 'Silver 3', 'Gold 1', 'Gold 2', 'Gold 3',
    'Platinum 1', 'Platinum 2', 'Platinum 3', 'Diamond 1', 'Diamond 2',
    'Diamond 3', 'Immortal 1', 'Immortal 2', 'Immortal 3', 'Radiant'
])})

# Clustering Analysis
features = ['Kills', 'Deaths', 'Assists', 'CombatScore', 'Headshot%', 'KDA', 'ADR']
X = ddf[features].to_dask_array(lengths=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)
clusters = kmeans.predict(X_scaled).compute()
ddf = ddf.assign(Cluster=dd.from_array(clusters))

# Process Data
df = ddf.compute()
current_rank = df['Rank'].iloc[-1]
df['Cluster'] = df['Cluster'].map({
    0: 'Aggressive Duelist',
    1: 'Strategic Leader',
    2: 'Support Specialist',
    3: 'Flex Playmaker'
})
playstyle = df['Cluster'].mode()[0]  # <-- ADD THIS LINE


# Enhanced Visualizations
def create_performance_timeline(df):
    fig = px.line(df, x='Date', y='RankScore',
                 title='Rank Progression Timeline',
                 color_discrete_sequence=['#FF4655'],
                 hover_data={'Rank': True, 'KDA': ':.2f', 'ADR': ':.1f'})

    fig.update_yaxes(
        title='Rank',
        tickvals=list(range(22)),
        ticktext=[
            'Iron 1', 'Iron 2', 'Iron 3', 'Bronze 1', 'Bronze 2', 'Bronze 3',
            'Silver 1', 'Silver 2', 'Silver 3', 'Gold 1', 'Gold 2', 'Gold 3',
            'Platinum 1', 'Platinum 2', 'Platinum 3', 'Diamond 1', 'Diamond 2',
            'Diamond 3', 'Immortal 1', 'Immortal 2', 'Immortal 3', 'Radiant'
        ]
    )

    # Add rolling average
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['RankScore'].rolling(10).mean(),
        name='10-Match Average',
        line=dict(color='#00FF9D', dash='dot')
    ))

    fig.update_layout(
        hovermode='x unified',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        xaxis_showgrid=False,
        yaxis_showgrid=False
    )

    return fig
def create_3d_radar(df):
    # Normalize features for radar plot
    features = ['Kills', 'Assists', 'CombatScore', 'Headshot%', 'ADR']
    scaled = (df[features] - df[features].min()) / (df[features].max() - df[features].min())

    fig = go.Figure()

    for cluster in df['Cluster'].unique():
        cluster_data = scaled[df['Cluster'] == cluster]
        fig.add_trace(go.Scatterpolar(
            r=cluster_data.mean().values,
            theta=features,
            fill='toself',
            name=cluster,
            hoverinfo='name+text',
            text=[f"Avg: {cluster_data[col].mean():.2f}" for col in features]
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1]),
            angularaxis=dict(linecolor='#FFFFFF')
        ),
        template='plotly_dark',
        title='3D Playstyle Radar Analysis',
        showlegend=True,
        height=700,
        scene=dict(
            aspectmode='cube',
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
    return fig

# Generate HTML Report
win_rate = df['Outcome'].eq('Win').mean() * 100
main_agent = df['Agent'].mode()[0]
best_map = df.groupby('Map')['Outcome'].apply(lambda x: (x == 'Win').mean()).idxmax()
avg_kda = df['KDA'].mean()
avg_hs = df['Headshot%'].mean()

html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>VALORANT PRO TRACKER - {player_id}</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Oxanium:wght@400;700&display=swap" rel="stylesheet">
    <style>
        :root {{
            --valorant-red: #FF4655;
            --valorant-black: #0F1923;
            --valorant-white: #ECE8E1;
        }}

        body {{
            font-family: 'Oxanium', sans-serif;
            background: var(--valorant-black);
            color: var(--valorant-white);
            margin: 0;
            padding: 2rem;
        }}

        .header {{
            text-align: center;
            padding: 3rem;
            background: linear-gradient(135deg, #1A1A1A 0%, #0F1923 100%);
            border-bottom: 3px solid var(--valorant-red);
            margin-bottom: 2rem;
        }}

        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
            gap: 1.5rem;
            margin-bottom: 3rem;
        }}

        .stat-card {{
            background: rgba(255, 70, 85, 0.1);
            padding: 1.5rem;
            border-radius: 12px;
            backdrop-filter: blur(10px);
            border: 1px solid rgba(255, 70, 85, 0.3);
            transition: transform 0.3s ease;
        }}

        .stat-card:hover {{
            transform: translateY(-5px);
        }}

        .chart-container {{
            background: rgba(255, 255, 255, 0.05);
            padding: 2rem;
            margin: 2rem 0;
            border-radius: 12px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
        }}

        h1, h2, h3 {{
            color: var(--valorant-red);
            margin: 0 0 1rem 0;
        }}

        .highlight {{
            color: var(--valorant-red);
            font-size: 2.5rem;
            font-weight: 700;
            text-shadow: 0 0 10px rgba(255, 70, 85, 0.5);
        }}

        .grid-2col {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 2rem;
            margin-top: 2rem;
        }}

        .match-history {{
            max-height: 400px;
            overflow-y: auto;
            border: 1px solid rgba(255, 70, 85, 0.2);
            border-radius: 8px;
        }}
    </style>
</head>
<body>
    <div class="header">
        <h1>VALORANT PERFORMANCE TRACKER</h1>
        <h2>{player_id}</h2>
        <h3>Current Rank: <span class="highlight">{current_rank}</span></h3>
    </div>

    <div class="stats-grid">
        <div class="stat-card">
            <h3>🏆 Win Rate</h3>
            <div class="highlight">{win_rate:.1f}%</div>
            <p>{len(df)} matches analyzed</p>
        </div>

        <div class="stat-card">
            <h3>🔫 Combat Mastery</h3>
            <p>Avg KDA: {avg_kda:.2f}</p>
            <p>Headshot %: {avg_hs:.1f}%</p>
            <p>ADR: {df['ADR'].mean():.1f}</p>
        </div>

        <div class="stat-card">
            <h3>🎮 Player Profile</h3>
            <div class="highlight" style="color: #00FF9D">{playstyle}</div>
            <p>Main Agent: {main_agent}</p>
            <p>Best Map: {best_map}</p>
            <p>Total Kills: {df['Kills'].sum()}</p>
        </div>
    </div>

    <div class="chart-container">
        <h3>📈 Rank Progression Timeline</h3>
        {create_performance_timeline(df).to_html(full_html=False, include_plotlyjs='cdn')}
    </div>

    <div class="grid-2col">
        <div class="chart-container">
            <h3>🎯 Playstyle Radar Analysis</h3>
            {create_3d_radar(df).to_html(full_html=False, include_plotlyjs='cdn')}
        </div>

        <div class="chart-container">
            <h3>🕹️ Agent Performance Matrix</h3>
            {px.treemap(df, path=['Role', 'Agent'], values='CombatScore',
                      color='KDA', color_continuous_scale='Viridis').to_html(full_html=False)}
        </div>
    </div>

    <div class="chart-container">
        <h3>📅 Recent Match History</h3>
        <div class="match-history">
            <table style="width:100%; border-collapse: collapse;">
                <tr style="background: var(--valorant-red);">
                    <th>Date</th>
                    <th>Agent</th>
                    <th>Map</th>
                    <th>K/D/A</th>
                    <th>Score</th>
                    <th>HS%</th>
                </tr>
                {"".join([f'''
                <tr style="border-bottom: 1px solid rgba(255,255,255,0.1);">
                    <td>{row.Date}</td>
                    <td>{row.Agent}</td>
                    <td>{row.Map}</td>
                    <td>{row.Kills}/{row.Deaths}/{row.Assists}</td>
                    <td>{row.CombatScore}</td>
                    <td>{row['Headshot%']}%</td>
                </tr>
                ''' for _, row in df.tail(10).iterrows()])}
            </table>
        </div>
    </div>
</body>
</html>
"""

report_filename = f"{player_id.replace('#', '_')}_pro_tracker.html"
with open(report_filename, "w") as f:
    f.write(html)

print(f"\n🔥 Professional Tracker Generated: {report_filename}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 5.7 MB/s eta 0:00:00
Enter your Valorant ID (e.g., Tenz#NA1): god


/usr/local/lib/python3.11/dist-packages/dask_expr/_collection.py:4225: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Rank', 'float64'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(



🔥 Professional Tracker Generated: god_pro_tracker.html
